In [1]:
%reload_ext autoreload
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Imports

In [2]:
# External libraries
import os as so
import sys as s
import pathlib as pl
import torch
import torch.nn as nn
import torch.optim as om
from torch import Tensor
from torch.utils.data import random_split
from torch.utils.data import DataLoader, ConcatDataset, Dataset
import torcheval
from torcheval.metrics import MulticlassF1Score, Mean
import optuna as opt
import torchvision as tn
import sklearn as sn
from sklearn.metrics import f1_score
import pandas as ps
import numpy as ny
import typing as t
import pathlib as pl
import matplotlib.pyplot as pt
import random as rng
from tqdm import tqdm
import tqdm as tm
from pprint import pprint
from git import Repo
import lightning as tl
from lightning.pytorch.loggers import WandbLogger

In [3]:
# Add local package to path
if (p := pl.Path(so.getcwd(), '..').absolute().as_posix()) not in s.path:
    s.path.append(p)

# Local imports
from gic import *
from gic.models.resnet import ResCNN
from gic.models.densenet import DenseCNN
from gic.models.convnext import ConvNextNet
from gic.models.autoencoder import AutoEncoder, UNet
from gic.data import load_data, load_batched_data, GICPreprocess, GICPerturb, GICDatasetModule

Seed set to 7982


### Data Loading

In [4]:
train_dl, valid_dl, test_dl = load_batched_data(DATA_PATH, 'disjoint', gen_torch, pin_memory=pin_memory, num_workers=num_workers, prefetch_factor=prefetch_factor, batch_size=32)
tr_perturb = GICPerturb(mask=True, normalize=False)
tr_preprocess = GICPreprocess(augment=False, normalize=False)
tr_augment = GICPreprocess(augment=True, normalize=False)
tr_norm = GICPreprocess(normalize=True, augment=False)

### Train Denoising UNet AutoEncoder

In [5]:
import wandb as wn
from typing import Any
from lightning.pytorch.utilities.types import STEP_OUTPUT, OptimizerLRScheduler


class AutoEncoderModule(tl.LightningModule):
    logger: WandbLogger

    def __init__(self, **kwargs):
        super(AutoEncoderModule, self).__init__()
        self.save_hyperparameters(kwargs, logger=True)

        # Model
        self.__loss_l1 = nn.L1Loss()
        self.__loss_l2 = nn.MSELoss()
        self.__unet = AutoEncoder(**kwargs)

        # Data Transformation Operations
        self.__tr_mask = GICPerturb(True, False)
        self.__tr_norm = GICPreprocess(augment=False, normalize=True)

        # Metrics
        self.__metric_loss_l1 = Mean(device=self.device)
        self.__metric_loss_l2 = Mean(device=self.device)
        self.__metric_loss_w = Mean(device=self.device)
        self.__top_k = 16

    def forward(self, x: Tensor) -> Tensor:
        return self.__unet(x)[0]

    def on_train_start(self) -> None:
        self.metrics_to_device()
        self.__tb_true: Tensor = torch.stack([self.dataset[i][0] for i in range(self.__top_k)])
        self.__tb_true_grid = tn.utils.make_grid(self.__tb_true, nrow=4, pad_value=5)
        self.__tb_mask: Tensor = self.__tr_mask(self.__tb_true)
        self.__tb_mask_grid = tn.utils.make_grid(self.__tb_mask, nrow=4, pad_value=5)

    def on_train_epoch_start(self) -> None:
        self.metrics_reset()

    def training_step(self, batch: t.Tuple[Tensor, Tensor], _: t.Any) -> STEP_OUTPUT:
        # Retrieve image => apply noise and mask => denoise
        X_raw: Tensor = batch[0]
        X_true: Tensor = self.__tr_norm(X_raw)
        X_noisy: Tensor = self.__tr_norm(self.__tr_mask(X_raw))
        X_pred: Tensor = self(X_noisy)

        # Reconstruct the original input
        loss_l1: Tensor = self.__loss_l1(X_pred, X_true)
        loss_l2: Tensor = self.__loss_l2(X_pred, X_true)
        loss: Tensor = 0.5 * loss_l1 + 0.5 * loss_l2

        # Update current metrics
        self.__metric_loss_l1.update(loss_l1.detach())
        self.__metric_loss_l2.update(loss_l2.detach())
        self.__metric_loss_w.update(loss.detach())
        return loss

    @torch.no_grad()
    def on_train_epoch_end(self) -> None:
        self.metrics_log()
        b_mask: Tensor = self.__tr_norm(self.__tb_mask.to(self.device))
        b_pred: Tensor = self.__tr_norm.denorm(self(b_mask).detach()).cpu()
        b_pred_grid = tn.utils.make_grid(b_pred, nrow=4, pad_value=5)
        img_a = tn.transforms.transforms.F.to_pil_image(self.__tb_true_grid)
        img_b = tn.transforms.transforms.F.to_pil_image(self.__tb_mask_grid)
        img_c = tn.transforms.transforms.F.to_pil_image(b_pred_grid)
        names = ['Original', 'Noisy', 'Denoised']
        images = [img_a, img_b, img_c]
        self.logger.log_image(key='Image Denoising', images=images, caption=names)

    def on_validation_start(self) -> None:
        self.metrics_to_device()
        self.__vb_true: Tensor = torch.stack([self.dataset[i][0] for i in range(self.__top_k)])
        self.__vb_true_grid = tn.utils.make_grid(self.__vb_true, nrow=4, pad_value=5)

    def on_validation_epoch_start(self) -> None:
        self.metrics_reset()

    def validation_step(self, batch: t.Tuple[Tensor, Tensor], _: t.Any) -> STEP_OUTPUT:
        # Retrieve image => apply noise and mask => denoise
        X_true: Tensor = self.__tr_norm(batch[0])
        X_pred: Tensor = self(X_true)

        # Reconstruct the original input
        loss_l1: Tensor = self.__loss_l1(X_pred, X_true)
        loss_l2: Tensor = self.__loss_l2(X_pred, X_true)
        loss: Tensor = 0.5 * loss_l1 + 0.5 * loss_l2

        # Update current metrics
        self.__metric_loss_l1.update(loss_l1)
        self.__metric_loss_l2.update(loss_l2)
        self.__metric_loss_w.update(loss)
        return loss

    def on_validation_epoch_end(self) -> None:
        self.metrics_log()
        b_true: Tensor = self.__tr_norm(self.__vb_true.to(self.device))
        b_pred: Tensor = self.__tr_norm.denorm(self(b_true)).cpu()
        b_pred_grid = tn.utils.make_grid(b_pred, nrow=4, pad_value=5)
        img_a = tn.transforms.transforms.F.to_pil_image(self.__vb_true_grid)
        img_b = tn.transforms.transforms.F.to_pil_image(b_pred_grid)
        names = ['Original', 'Reconstructed']
        images= [img_a, img_b]
        self.logger.log_image(key='Image Reconstruction', images=images, caption=names)

    def test_step(self, batch: Tensor, _: t.Any) -> STEP_OUTPUT:
        return self(batch)

    def predict_step(self, batch: Tensor, _: t.Any) -> Any:
        return self(batch)

    def configure_optimizers(self) -> OptimizerLRScheduler:
        return om.AdamW(self.parameters(), betas=(0.9, 0.999), lr=6e-4)

    def metrics_log(self):
        self.log(f'{self.mode}_loss_l1', self.__metric_loss_l1.compute().item())
        self.log(f'{self.mode}_loss_l2', self.__metric_loss_l2.compute().item())
        self.log(f'{self.mode}_loss_w', self.__metric_loss_w.compute().item())

    def metrics_to_device(self):
        self.__metric_loss_l1.to(self.device)
        self.__metric_loss_l2.to(self.device)
        self.__metric_loss_w.to(self.device)

    def metrics_reset(self):
        self.__metric_loss_l1.reset()
        self.__metric_loss_l2.reset()
        self.__metric_loss_w.reset()

    @property
    def dataset(self) -> Dataset:
        return self.dataloader.dataset

    @property
    def dataloader(self) -> DataLoader:
        if self.trainer.training:
            return t.cast(DataLoader, self.trainer.train_dataloader)
        elif self.trainer.validating:
            return t.cast(DataLoader, self.trainer.val_dataloaders)
        elif self.trainer.testing:
            return t.cast(DataLoader, self.trainer.test_dataloaders)
        else:
            return t.cast(DataLoader, self.trainer.predict_dataloaders)

    @property
    def mode(self):
        if self.trainer.training:
            return 'train'
        elif self.trainer.validating:
            return 'valid'
        elif self.trainer.testing:
            return 'test'
        else:
            return 'pred'

In [6]:
wb_logger = WandbLogger(project='Generated Image Classification', name='UNet Test', save_dir=LOG_PATH)
trainer = tl.Trainer(max_epochs=20, logger=wb_logger, num_sanity_val_steps=0)
model = AutoEncoderModule(chan=64, hchan=256, activ_fn='SiLU')

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [7]:
trainer.fit(model, train_dataloaders=train_dl, val_dataloaders=valid_dl)

wandb: Currently logged in as: invokariman (castelvaar). Use `wandb login --relogin` to force relogin


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name                        | Type          | Params
--------------------------------------------------------------
0 | _AutoEncoderModule__loss_l1 | L1Loss        | 0     
1 | _AutoEncoderModule__loss_l2 | MSELoss       | 0     
2 | _AutoEncoderModule__unet    | AutoEncoder   | 26.1 M
3 | _AutoEncoderModule__tr_mask | GICPerturb    | 0     
4 | _AutoEncoderModule__tr_norm | GICPreprocess | 0     
--------------------------------------------------------------
26.1 M    Trainable params
0         Non-trainable params
26.1 M    Total params
104.208   Total estimated model params size (MB)
/home/invokariman/.cache/pypoetry/virtualenvs/gic-BfYgXNhZ-py3.11/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=31` in the `DataLoader` to improve performance.
/h

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=20` reached.


In [12]:
torch.save(model.state_dict(), CKPT_PATH / 'unet.pt')

In [6]:
model = AutoEncoderModule(chan=64, hchan=256, activ_fn='SiLU').to(DEVICE)
model.load_state_dict(torch.load(CKPT_PATH / 'unet.pt'))
model.eval().requires_grad_(False)

AutoEncoderModule(
  (_AutoEncoderModule__loss_l1): L1Loss()
  (_AutoEncoderModule__loss_l2): MSELoss()
  (_AutoEncoderModule__unet): AutoEncoder(
    (unet): UNet(
      (encoder): Encoder(
        (in_layer): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (layers): ModuleList(
          (0): DownSampleBlock(
            (conv_layer): ConvBlock(
              (depthwise_layer): Sequential(
                (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64)
                (1): LayerNorm((64, 64, 64), eps=1e-05, elementwise_affine=True)
              )
              (pointwise_layer): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1))
              (activ_fn): SiLU()
              (bottleneck_layer): Conv2d(256, 64, kernel_size=(1, 1), stride=(1, 1))
            )
            (down_layer): Conv2d(64, 128, kernel_size=(2, 2), stride=(2, 2), bias=False)
          )
          (1): DownSampleBlock(
            (conv_layer): ConvBlock(

In [7]:
tr_embd = [model._AutoEncoderModule__unet.forward(tr_norm(X.to(DEVICE)))[1].cpu() for X, _ in iter(train_dl)]
tr_embd = torch.cat(tr_embd, dim=0).flatten(start_dim=1).numpy()

In [8]:
val_embd = [model._AutoEncoderModule__unet.forward(tr_norm(X.to(DEVICE)))[1].cpu() for X, _ in iter(valid_dl)]
val_embd = torch.cat(val_embd, dim=0).flatten(start_dim=1).numpy()

In [9]:
test_embd = [model._AutoEncoderModule__unet.forward(tr_norm(X.to(DEVICE)))[1].cpu() for X in iter(test_dl)]
test_embd = torch.cat(test_embd, dim=0).flatten(start_dim=1).numpy()

In [10]:
tr_y = torch.cat([y for _, y in iter(train_dl)]).numpy()
vl_y = torch.cat([y for _, y in iter(valid_dl)]).numpy()

In [11]:
import sklearn as kl
from sklearn.svm import LinearSVC


svm = LinearSVC(C=1.0)

In [12]:
svm.fit(tr_embd, tr_y)

/home/invokariman/.cache/pypoetry/virtualenvs/gic-BfYgXNhZ-py3.11/lib/python3.11/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


### Model

In [8]:
from typing import Any
from lightning.pytorch.utilities.types import STEP_OUTPUT, OptimizerLRScheduler


class DenseNetModule(tl.LightningModule):
    def __init__(self, **kwargs):
        super(DenseNetModule, self).__init__()

        # Model
        unet = AutoEncoder(64, 256, 'SiLU')
        unet.load_state_dict(torch.load('../ckpt/unet.pt'))
        unet.eval().requires_grad_(False)
        self.enc = unet.unet.encoder
        f_drop = 0.2

        self.net_densecnn = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),

            nn.Dropout1d(f_drop),
            nn.Linear(in_features=1024, out_features=512),
            nn.BatchNorm1d(512),
            nn.SiLU(),

            nn.Dropout1d(f_drop),
            nn.Linear(in_features=512, out_features=512),
            nn.BatchNorm1d(512),
            nn.SiLU(),

            nn.Linear(in_features=512, out_features=100)
        )
        # self.net_densecnn= ConvNextNet(64, 4, conv_dropout=0.1, dense_dropout=0.3, conv_layers=2, dense_features=256, dense_layers=1, patch_reduce=False)
        # self.net_densecnn = DenseCNN(**kwargs)
        self.loss_fn = nn.CrossEntropyLoss()

        # Metrics
        self.metric_train_f1_score = MulticlassF1Score(num_classes=CONST_NUM_CLASS, average='macro', device=self.device)
        self.metric_valid_f1_score = MulticlassF1Score(num_classes=CONST_NUM_CLASS, average='macro', device=self.device)
        self.metric_train_loss = Mean(device=self.device)
        self.metric_valid_loss = Mean(device=self.device)
        self.save_hyperparameters(kwargs)

    def forward(self, x: Tensor) -> Tensor:
        print(x.shape)
        _, e = self.enc(x)
        return self.net_densecnn(e)

    def on_train_start(self) -> None:
        self.metric_train_f1_score.to(self.device)
        self.metric_train_loss.to(self.device)

    def on_train_epoch_start(self) -> None:
        self.metric_train_f1_score.reset()
        self.metric_train_loss.reset()

    def training_step(self, batch: t.Tuple[Tensor, Tensor], _: t.Any) -> STEP_OUTPUT:
        X, y_true = batch
        logits: Tensor = self(X)
        loss: Tensor = self.loss_fn(logits, y_true)

        self.metric_train_loss.update(loss.detach())
        self.metric_train_f1_score.update(logits.detach(), y_true)
        return loss

    def on_train_epoch_end(self) -> None:
        self.log('train_f1_score', self.metric_train_f1_score.compute().item())
        self.log('train_loss', self.metric_train_loss.compute().item())

    def on_validation_start(self) -> None:
        self.metric_valid_loss.to(self.device)
        self.metric_valid_f1_score.to(self.device)

    def on_validation_epoch_start(self) -> None:
        self.metric_valid_f1_score.reset()
        self.metric_valid_loss.reset()

    def validation_step(self, batch: t.Tuple[Tensor, Tensor], _: t.Any) -> STEP_OUTPUT:
        X, y_true = batch
        logits: Tensor = self(X)
        loss: Tensor = self.loss_fn(logits, y_true)

        self.metric_valid_loss.update(loss)
        self.metric_valid_f1_score.update(logits, y_true)
        return loss

    def on_validation_epoch_end(self) -> None:
        self.log('valid_f1_score', self.metric_valid_f1_score.compute().item())
        self.log('valid_loss', self.metric_valid_loss.compute().item())

    def predict_step(self, batch: Tensor, _: t.Any) -> Any:
        return torch.argmax(self(batch), dim=-1)

    def configure_optimizers(self) -> OptimizerLRScheduler:
        optim = om.AdamW(self.parameters(), betas=(0.9, 0.999), lr=6e-4)
        scheduler = om.lr_scheduler.ReduceLROnPlateau(optim, 'max', 0.75, 10, min_lr=2e-4, cooldown=5)
        return {
            'optimizer': optim,
            'lr_scheduler': {
                'scheduler': scheduler,
                'monitor': 'train_f1_score',
            },
        }

### Logging

In [9]:
gic_data = GICDatasetModule(DATA_PATH, False, 32, num_workers, prefetch_factor, pin_memory, True, gen_torch)
trainer = tl.Trainer(max_epochs=150, enable_checkpointing=True, logger=wb_logger)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


### Training & Validation Loop

In [10]:
net_densecnn = DenseNetModule()
trainer.fit(net_densecnn, datamodule=gic_data)

/home/invokariman/.cache/pypoetry/virtualenvs/gic-BfYgXNhZ-py3.11/lib/python3.11/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:639: Checkpoint directory ../log/Generated Image Classification/zncf9ya2/checkpoints exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type             | Params
--------------------------------------------------
0 | enc          | Encoder          | 6.6 M 
1 | net_densecnn | Sequential       | 840 K 
2 | loss_fn      | CrossEntropyLoss | 0     
--------------------------------------------------
840 K     Trainable params
6.6 M     Non-trainable params
7.4 M     Total params
29.642    Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/invokariman/.cache/pypoetry/virtualenvs/gic-BfYgXNhZ-py3.11/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=31` in the `DataLoader` to improve performance.


torch.Size([32, 3, 64, 64])


AttributeError: 'list' object has no attribute 'size'

### Full Training Loop

In [ ]:
gic_data = GICDatasetModule(DATA_PATH, True, 32, num_workers, prefetch_factor, pin_memory, False, gen_torch)
trainer = tl.Trainer(max_epochs=345, enable_checkpointing=False, logger=wb_logger)
net_densecnn = DenseNetModule()
trainer.fit(net_densecnn, datamodule=gic_data)

### Testing Loop

In [ ]:
y_hat: t.List[Tensor] = t.cast(t.List[Tensor], trainer.predict(net_densecnn, datamodule=gic_data, return_predictions=True))
preds = torch.cat(y_hat, dim=0)

### Submission

In [ ]:
data = test_dl.dataset._GICDataset__data
data['Class'] = preds
data.to_csv(SUBMISSION_PATH, index=False)